## Top 10 arrival airports in the world in 2013 (using the bookings file)

Arrival airport is the column arr_port. It is the IATA code for the airport

To get the total number of passengers for an airport, you can sum the column pax, grouping by arr_port.

Note that there is negative pax. That corresponds to cancelations. So to get the total number of passengers that have actually booked, you should sum including the negatives (that will remove the canceled bookings).

Print the top 10 arrival airports in the standard output, including the number of passengers.

Bonus point: Get the name of the city or airport corresponding to that airport (programatically, we suggest to have a look at GeoBases in Github)

Bonus point: Solve this problem using pandas (instead of any other approach)


## 0) Prepare booking Sample (we might need this for later)

In [1]:
!pwd

/home/dsc/Data/DS_Master_A1_Chall


In [2]:
cd ../challenge/

/home/dsc/Data/challenge


In [4]:
ls -l

total 1013840
-rw-rw-r--. 1 dsc dsc 554970628 Jan  4 15:59 bookings.csv.bz2
-rw-rw-r--. 1 dsc dsc         0 May  5 11:58 bookings.sample.csv
-rw-rw-r--. 1 dsc dsc 483188920 Jan  4 16:01 searches.csv.bz2


In [5]:
! bzcat bookings.csv.bz2 | head -10000 > bookings.sample.csv


bzcat: I/O or other error, bailing out.  Possible reason follows.
bzcat: Broken pipe
	Input file = bookings.csv.bz2, output file = (stdout)


In [6]:
! wc -l  bookings.sample.csv

10000 bookings.sample.csv


In [7]:
! bzip2 bookings.sample.csv

In [8]:
! bzcat bookings.sample.csv.bz2 | wc -l

10000


## 1) Get familiar with data

In [9]:
import pandas as pd

### What if we dont want to read the whole file?

In [22]:
b = pd.read_csv('bookings.sample.csv.bz2',sep='^')

In [10]:
b=pd.read_csv('bookings.csv.bz2',sep='^', nrows=1000)

In [ ]:
b.info()

In [47]:
b.head()

,act_date,source,pos_ctry,pos_iata,pos_oid,rloc,cre_date,duration,distance,dep_port,...,route,carrier,bkg_class,cab_class,brd_time,off_time,pax,year,month,oid
0,2013-03-05 00:00:00,1A,DE,a68dd7ae953c8acfb187a1af2dcbe123,1a11ae49fcbf545fd2afc1a24d88d2b7,ea65900e72d71f4626378e2ebd298267,2013-02-22 00:00:00,1708,0,ZRH,...,LHRZRH,VI,T,Y,2013-03-07 08:50:00,2013-03-07 11:33:37,-1,2013,3,NULL
1,2013-03-26 00:00:00,1A,US,e612b9eeeee6f17f42d9b0d3b79e75ca,7437560d8f276d6d05eeb806d9e7edee,737295a86982c941f1c2da9a46a14043,2013-03-26 00:00:00,135270,0,SAL,...,SALATLCLT,NV,L,Y,2013-04-12 13:04:00,2013-04-12 22:05:40,1,2013,3,NULL
2,2013-03-26 00:00:00,1A,US,e612b9eeeee6f17f42d9b0d3b79e75ca,7437560d8f276d6d05eeb806d9e7edee,737295a86982c941f1c2da9a46a14043,2013-03-26 00:00:00,135270,0,SAL,...,CLTATLSAL,NV,U,Y,2013-07-15 07:00:00,2013-07-15 11:34:51,1,2013,3,NULL
3,2013-03-26 00:00:00,1A,AU,0f984b3bb6bd06661c95529bbd6193bc,36472c6dbaf7afec9136ac40364e2794,5ecf00fdcbcec761c43dc7285253d0c1,2013-03-26 00:00:00,30885,0,AKL,...,AKLHKGSVO,XK,G,Y,2013-04-24 23:59:00,2013-04-25 16:06:31,1,2013,3,SYDA82546
4,2013-03-26 00:00:00,1A,AU,0f984b3bb6bd06661c95529bbd6193bc,36472c6dbaf7afec9136ac40364e2794,5ecf00fdcbcec761c43dc7285253d0c1,2013-03-26 00:00:00,30885,0,AKL,...,SVOHKGAKL,XK,G,Y,2013-05-14 20:15:00,2013-05-16 10:44:50,1,2013,3,SYDA82546


In [70]:
b.columns

Index([u'act_date           ', u'source', u'pos_ctry', u'pos_iata',
       u'pos_oid  ', u'rloc          ', u'cre_date           ', u'duration',
       u'distance', u'dep_port', u'dep_city', u'dep_ctry', u'arr_port',
       u'arr_city', u'arr_ctry', u'lst_port', u'lst_city', u'lst_ctry',
       u'brd_port', u'brd_city', u'brd_ctry', u'off_port', u'off_city',
       u'off_ctry', u'mkt_port', u'mkt_city', u'mkt_ctry', u'intl',
       u'route          ', u'carrier', u'bkg_class', u'cab_class',
       u'brd_time           ', u'off_time           ', u'pax', u'year',
       u'month', u'oid      '],
      dtype='object')

In [72]:
b.columns.tolist()

['act_date           ',
 'source',
 'pos_ctry',
 'pos_iata',
 'pos_oid  ',
 'rloc          ',
 'cre_date           ',
 'duration',
 'distance',
 'dep_port',
 'dep_city',
 'dep_ctry',
 'arr_port',
 'arr_city',
 'arr_ctry',
 'lst_port',
 'lst_city',
 'lst_ctry',
 'brd_port',
 'brd_city',
 'brd_ctry',
 'off_port',
 'off_city',
 'off_ctry',
 'mkt_port',
 'mkt_city',
 'mkt_ctry',
 'intl',
 'route          ',
 'carrier',
 'bkg_class',
 'cab_class',
 'brd_time           ',
 'off_time           ',
 'pax',
 'year',
 'month',
 'oid      ']

In [25]:
list(b.columns)

['act_date           ',
 'source',
 'pos_ctry',
 'pos_iata',
 'pos_oid  ',
 'rloc          ',
 'cre_date           ',
 'duration',
 'distance',
 'dep_port',
 'dep_city',
 'dep_ctry',
 'arr_port',
 'arr_city',
 'arr_ctry',
 'lst_port',
 'lst_city',
 'lst_ctry',
 'brd_port',
 'brd_city',
 'brd_ctry',
 'off_port',
 'off_city',
 'off_ctry',
 'mkt_port',
 'mkt_city',
 'mkt_ctry',
 'intl',
 'route          ',
 'carrier',
 'bkg_class',
 'cab_class',
 'brd_time           ',
 'off_time           ',
 'pax',
 'year',
 'month',
 'oid      ']

In [27]:
b.shape

(1000, 38)

In [57]:
description=b.describe()

In [58]:
description

,duration,distance,intl,pax,year,month
count,1000.000000,1000.000000,1000.000000,1000.000000,1000,1000
mean,23935.433000,407.569000,0.643000,0.532000,2013,3
std,54459.101785,1490.087967,0.479355,1.976077,0,0
min,61.000000,0.000000,0.000000,-8.000000,2013,3
25%,3493.000000,0.000000,0.000000,-1.000000,2013,3
50%,6891.000000,0.000000,1.000000,1.000000,2013,3
75%,17940.000000,0.000000,1.000000,1.000000,2013,3
max,482174.000000,10995.000000,1.000000,21.000000,2013,3


In [73]:
b.describe(include='all')

,act_date,source,pos_ctry,pos_iata,pos_oid,rloc,cre_date,duration,distance,dep_port,dep_city,dep_ctry,arr_port,arr_city,arr_ctry,lst_port,lst_city,lst_ctry,brd_port,brd_city,brd_ctry,off_port,off_city,off_ctry,mkt_port,mkt_city,mkt_ctry,intl,route,carrier,bkg_class,cab_class,brd_time,off_time,pax,year,month,oid
count,1000,1000,1000,1000,1000,1000,1000,1000.000000,1000.000000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000.000000,1000,1000,1000,1000,1000,1000,1000.000000,1000,1000,1000
unique,31,5,53,278,281,352,57,NaN,NaN,176,166,57,201,195,74,184,175,60,263,250,85,267,253,83,403,395,197,NaN,634,86,26,3,688,704,NaN,NaN,NaN,45
top,2013-03-06 00:00:00,1P,US,83ccf07a55606976f78e63fcc805f58c,1490c3c5f95f04fa076fe15c775a80a9,a37584d1485cb35991e4ff1a2ba92262,2013-03-04 00:00:00,NaN,NaN,LAX,NYC,US,MNL,NYC,US,ORD,NYC,US,LHR,NYC,US,LHR,NYC,US,CCJJED,CCJJED,USUS,NaN,OSLISTISU,FK,V,Y,2013-03-10 18:20:00,2013-03-11 03:52:53,NaN,NaN,NaN,NULL
freq,75,372,402,59,59,12,84,NaN,NaN,30,42,389,20,36,334,30,47,394,27,35,352,23,44,365,11,11,270,NaN,6,122,80,897,6,6,NaN,NaN,NaN,861
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23935.433000,407.569000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.643000,NaN,NaN,NaN,NaN,NaN,NaN,0.532000,2013,3,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,54459.101785,1490.087967,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.479355,NaN,NaN,NaN,NaN,NaN,NaN,1.976077,0,0,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,-8.000000,2013,3,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3493.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,-1.000000,2013,3,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6891.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,2013,3,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17940.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,2013,3,NaN


In [28]:
b.isnull()

,act_date,source,pos_ctry,pos_iata,pos_oid,rloc,cre_date,duration,distance,dep_port,...,route,carrier,bkg_class,cab_class,brd_time,off_time,pax,year,month,oid
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
6,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
7,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
8,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
9,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [87]:
b.isnull().sum()

act_date               0
source                 0
pos_ctry               0
pos_iata               0
pos_oid                0
rloc                   0
cre_date               0
duration               0
distance               0
dep_port               0
dep_city               0
dep_ctry               0
arr_port               0
arr_city               0
arr_ctry               0
lst_port               0
lst_city               0
lst_ctry               0
brd_port               0
brd_city               0
brd_ctry               0
off_port               0
off_city               0
off_ctry               0
mkt_port               0
mkt_city               0
mkt_ctry               0
intl                   0
route                  0
carrier                0
bkg_class              0
cab_class              0
brd_time               0
off_time               0
pax                    0
year                   0
month                  0
oid                    0
dtype: int64

In [60]:
non_null_counts = b.count()

In [61]:
non_null_counts

act_date               1000
source                 1000
pos_ctry               1000
pos_iata               1000
pos_oid                1000
rloc                   1000
cre_date               1000
duration               1000
distance               1000
dep_port               1000
dep_city               1000
dep_ctry               1000
arr_port               1000
arr_city               1000
arr_ctry               1000
lst_port               1000
lst_city               1000
lst_ctry               1000
brd_port               1000
brd_city               1000
brd_ctry               1000
off_port               1000
off_city               1000
off_ctry               1000
mkt_port               1000
mkt_city               1000
mkt_ctry               1000
intl                   1000
route                  1000
carrier                1000
bkg_class              1000
cab_class              1000
brd_time               1000
off_time               1000
pax                    1000
year                

In [67]:
#pd.set_option('display.max_columns', 100)
pd.set_option('display.max_columns', None)


In [68]:
b.head()

,act_date,source,pos_ctry,pos_iata,pos_oid,rloc,cre_date,duration,distance,dep_port,dep_city,dep_ctry,arr_port,arr_city,arr_ctry,lst_port,lst_city,lst_ctry,brd_port,brd_city,brd_ctry,off_port,off_city,off_ctry,mkt_port,mkt_city,mkt_ctry,intl,route,carrier,bkg_class,cab_class,brd_time,off_time,pax,year,month,oid
0,2013-03-05 00:00:00,1A,DE,a68dd7ae953c8acfb187a1af2dcbe123,1a11ae49fcbf545fd2afc1a24d88d2b7,ea65900e72d71f4626378e2ebd298267,2013-02-22 00:00:00,1708,0,ZRH,ZRH,CH,LHR,LON,GB,ZRH,ZRH,CH,LHR,LON,GB,ZRH,ZRH,CH,LHRZRH,LONZRH,CHGB,1,LHRZRH,VI,T,Y,2013-03-07 08:50:00,2013-03-07 11:33:37,-1,2013,3,NULL
1,2013-03-26 00:00:00,1A,US,e612b9eeeee6f17f42d9b0d3b79e75ca,7437560d8f276d6d05eeb806d9e7edee,737295a86982c941f1c2da9a46a14043,2013-03-26 00:00:00,135270,0,SAL,SAL,SV,CLT,CLT,US,SAL,SAL,SV,SAL,SAL,SV,CLT,CLT,US,CLTSAL,CLTSAL,SVUS,1,SALATLCLT,NV,L,Y,2013-04-12 13:04:00,2013-04-12 22:05:40,1,2013,3,NULL
2,2013-03-26 00:00:00,1A,US,e612b9eeeee6f17f42d9b0d3b79e75ca,7437560d8f276d6d05eeb806d9e7edee,737295a86982c941f1c2da9a46a14043,2013-03-26 00:00:00,135270,0,SAL,SAL,SV,CLT,CLT,US,SAL,SAL,SV,CLT,CLT,US,SAL,SAL,SV,CLTSAL,CLTSAL,SVUS,1,CLTATLSAL,NV,U,Y,2013-07-15 07:00:00,2013-07-15 11:34:51,1,2013,3,NULL
3,2013-03-26 00:00:00,1A,AU,0f984b3bb6bd06661c95529bbd6193bc,36472c6dbaf7afec9136ac40364e2794,5ecf00fdcbcec761c43dc7285253d0c1,2013-03-26 00:00:00,30885,0,AKL,AKL,NZ,SVO,MOW,RU,AKL,AKL,NZ,AKL,AKL,NZ,SVO,MOW,RU,AKLSVO,AKLMOW,NZRU,1,AKLHKGSVO,XK,G,Y,2013-04-24 23:59:00,2013-04-25 16:06:31,1,2013,3,SYDA82546
4,2013-03-26 00:00:00,1A,AU,0f984b3bb6bd06661c95529bbd6193bc,36472c6dbaf7afec9136ac40364e2794,5ecf00fdcbcec761c43dc7285253d0c1,2013-03-26 00:00:00,30885,0,AKL,AKL,NZ,SVO,MOW,RU,AKL,AKL,NZ,SVO,MOW,RU,AKL,AKL,NZ,AKLSVO,AKLMOW,NZRU,1,SVOHKGAKL,XK,G,Y,2013-05-14 20:15:00,2013-05-16 10:44:50,1,2013,3,SYDA82546


## 2) Select the columns of interest 

In [11]:
b = pd.read_csv('bookings.csv.bz2',sep='^',usecols=['arr_port','pax', 'year'], nrows=1000)

In [12]:
b.head()

,arr_port,pax,year
0,LHR,-1,2013
1,CLT,1,2013
2,CLT,1,2013
3,SVO,1,2013
4,SVO,1,2013


## 3) What to do with NaN?



In [31]:
b.isnull().sum()

arr_port    0
pax         0
dtype: int64

In [32]:
b.count()

arr_port    1000
pax         1000
dtype: int64

In [94]:
b=b.dropna()

### What does df.dropna() do?

In [34]:
import numpy as np
df = pd.DataFrame(np.random.randn(5, 2), 
                  columns=['one', 'two' ])
df

,one,two
0,-0.587841,0.171796
1,-0.158598,0.574766
2,-1.058922,-0.103256
3,-0.135224,-0.919299
4,-0.059062,-0.718994


In [35]:
df['three']=[None, None, 1, 1, None]
df

,one,two,three
0,-0.587841,0.171796,NaN
1,-0.158598,0.574766,NaN
2,-1.058922,-0.103256,1
3,-0.135224,-0.919299,1
4,-0.059062,-0.718994,NaN


In [45]:
df.dropna()

,one,two,three
2,-1.058922,-0.103256,1
3,-0.135224,-0.919299,1


#### What if we want to drop just based on second column NaN?

In [44]:
df['two'].isnull()

0    False
1    False
2    False
3    False
4    False
Name: two, dtype: bool

In [55]:
~df['two'].isnull()

0    True
1    True
2    True
3    True
4    True
Name: two, dtype: bool

In [54]:
df[~(df['two'].isnull())]

,one,two,three
0,-0.587841,0.171796,NaN
1,-0.158598,0.574766,NaN
2,-1.058922,-0.103256,1
3,-0.135224,-0.919299,1
4,-0.059062,-0.718994,NaN


## 4) Ready to code

What do we have so far?

In [62]:
import pandas as pd
b = pd.read_csv('bookings.csv.bz2',sep='^',usecols=['arr_port','pax','year'], nrows=1000)
b=b.dropna()

In [63]:
len(b.dropna())

1000

In [64]:
b.shape

(1000, 3)

In [65]:
b.head()

,arr_port,pax,year
0,LHR,-1,2013
1,CLT,1,2013
2,CLT,1,2013
3,SVO,1,2013
4,SVO,1,2013


## 4) make action plan
1) get only the bookings from 2013

2) group by arr_port

3) sort 

4) get top 10

#### 4.1) Get only the booking from 2013

In [66]:
b = b[b['year'] == 2013]

#### 4.2) group by arr_port

In [69]:
arr_ports=b.groupby('arr_port')

In [72]:
for airport, val in arr_ports:
    print "%s" %(airport)
    print val['pax'].sum()

ACC     
6
ADD     
2
AGP     
6
ALA     
2
ALC     
4
ALG     
0
AMD     
1
AMS     
1
ANC     
0
ASU     
-10
ATH     
4
ATL     
3
ATQ     
1
AUH     
0
AUS     
2
AYT     
0
BBI     
0
BCN     
5
BEY     
4
BHM     
-1
BJV     
8
BJX     
2
BKK     
2
BLQ     
2
BLR     
0
BNA     
4
BNE     
6
BOG     
0
BOI     
3
BOM     
5
BOS     
11
BRD     
2
BRU     
2
BUD     
0
BUF     
2
BWI     
10
CAG     
0
CAI     
2
CAN     
0
CBR     
0
CCJ     
-3
CCS     
0
CDG     
4
CGK     
12
CHQ     
0
CLT     
5
CMB     
0
COK     
7
COS     
6
CTA     
0
CUN     
4
CUR     
0
CUZ     
0
CVG     
2
CZM     
4
DAB     
2
DAC     
4
DAY     
2
DCA     
50
DEL     
7
DEN     
6
DFW     
-2
DMM     
2
DOH     
3
DTW     
3
DXB     
0
ELP     
0
EWR     
8
EZE     
2
FAI     
4
FAO     
2
FCO     
3
FLL     
13
FRA     
4
FUE     
0
GAU     
-3
GIZ     
0
GRU     
0
GUM     
2
HAV     
0
HER     
4
HKG     
2
HMO     
1
HNL     
0
ICN     
0
IND     
3
ISB     
0
IST     
7
ISU     
0
IXE     
0

In [ ]:
for i, (airport, val) in enumerate(arr_ports):
    print "%s" %(airport)
    print val['pax'].sum()
    if (i==10) :break

ACC     
6
ADD     
2
AGP     
6
ALA     
2
ALC     
4
ALG     
0
AMD     
1
AMS     
1
ANC     
0
ASU     
-10
ATH     
4


In [164]:
arr_ports.sum().sort_values('pax', ascending=False).head()

,pax
arr_port,
DCA,50
JFK,14
FLL,13
PVG,13
CGK,12


In [162]:
topAirports = b.dropna().groupby(['arr_port']).sum().sort_values('pax', ascending=False).head(10)
topAirports

,pax
arr_port,
DCA,50
JFK,14
FLL,13
PVG,13
CGK,12
LGA,12
SVO,12
LAS,11
BOS,11


#### but... we have to read the whole file... and with nrows we are reading ALWAYS the first N rows...


In [7]:
%%time
b=pd.read_csv('bookings.sample.csv.bz2',sep='^',usecols=['arr_port','pax'], iterator=True)
b0=b.get_chunk(6000)
b1=b.get_chunk(3000)
b2=b.get_chunk(1001)
#b2=b.get_chunk(1)

print b0.head()
print b1
print b2


   arr_port  pax
0  LHR        -1
1  CLT         1
2  CLT         1
3  SVO         1
4  SVO         1
      arr_port  pax
0     BOS         1
1     BOS        -1
2     BOS        -1
3     BOS         1
4     PHX         1
5     GCC         1
6     GCC         1
7     JFK         1
8     JFK         1
9     LGA         1
10    LGA         1
11    LGA        -1
12    LGA         1
13    TAO         1
14    TAO         1
15    TAO        -1
16    TAO         1
17    YVR         1
18    MHT         1
19    DCA         1
20    DCA         1
21    GCM         1
22    GCM         1
23    ABQ         1
24    ABQ         1
25    ABQ        -1
26    ABQ         1
27    MIA         1
28    MIA         1
29    SJC        -2
...        ...  ...
2970  BGO         2
2971  BGO         2
2972  NAP         1
2973  ATL         1
2974  ATL         1
2975  ATL         1
2976  ATL         1
2977  ATL        -1
2978  ATL        -1
2979  DEN         1
2980  SLL         1
2981  SLL         1
2982  MCT         

In [5]:
bc = pd.read_csv('bookings.sample.csv.bz2',sep='^',usecols=['arr_port','pax'], chunksize=10)


In [6]:
%%time
bc = pd.read_csv('bookings.sample.csv.bz2',sep='^',usecols=['arr_port','pax'], chunksize=4000)
for i,chunk in enumerate(bc):
    chunk_airports=chunk.dropna().groupby(['arr_port']).sum().reset_index().head()
    print "Chank Nummber:%d" % i
    print chunk_airports


Chank Nummber:0
   arr_port  pax
0  ABQ         4
1  ABV         0
2  ABZ        -2
3  ACC         0
4  ADD         4
Chank Nummber:1
   arr_port  pax
0  AAL         0
1  ABQ         8
2  ABV         2
3  ABZ        -4
4  ACC         2
Chank Nummber:2
   arr_port  pax
0  ACC         2
1  ADZ         2
2  AEP         2
3  AFA         2
4  AJA         4
CPU times: user 269 ms, sys: 40.7 ms, total: 310 ms
Wall time: 367 ms


#### How do we get the results at one place?


In [225]:
sum_chunks = pd.read_csv('bookings.sample.csv.bz2',sep='^',usecols=['arr_port','pax'], nrows=0)

In [239]:
sum_chunks = pd.read_csv('bookings.sample.csv.bz2',sep='^',usecols=['arr_port','pax'], nrows=0)
bc = pd.read_csv('bookings.sample.csv.bz2',sep='^',usecols=['arr_port','pax'], chunksize=4000)
for i,chunk in enumerate(bc):
    chunk_airports=chunk.dropna().groupby(['arr_port']).sum().reset_index().head()
    #chunk_airports=chunk.dropna().groupby(['arr_port']).sum().head() #if we dont reset index arr_port is THe index
    #print chunk_airports
    sum_chunks=sum_chunks.append(chunk_airports)
    #sum_chunks=sum_chunks.append(chunk_airports, ignore_index=True)
sum_chunks


,arr_port,pax
0,ABQ,4
1,ABV,0
2,ABZ,-2
3,ACC,0
4,ADD,4
0,AAL,0
1,ABQ,8
2,ABV,2
3,ABZ,-4
4,ACC,2


In [9]:
#sum_chunks = pd.read_csv('bookings.sample.csv.bz2',sep='^',usecols=['arr_port','pax'], nrows=0)
sum_chunks=pd.DataFrame()
bc = pd.read_csv('bookings.sample.csv.bz2',sep='^',usecols=['arr_port','pax'], chunksize=4000)
for i,chunk in enumerate(bc):
    chunk_airports=chunk.dropna().groupby(['arr_port']).sum().reset_index().head()
    #chunk_airports=chunk.dropna().groupby(['arr_port']).sum().head() #if we dont reset index arr_port is THe index
    #print chunk_airports
    sum_chunks=pd.concat([sum_chunks,chunk_airports])
    #sum_chunks=sum_chunks.append(chunk_airports, ignore_index=True)
sum_chunks

,arr_port,pax
0,ABQ,4
1,ABV,0
2,ABZ,-2
3,ACC,0
4,ADD,4
0,AAL,0
1,ABQ,8
2,ABV,2
3,ABZ,-4
4,ACC,2


 ### We get the result here

In [10]:
#sum_chunks = pd.read_csv('bookings.sample.csv.bz2',sep='^',usecols=['arr_port','pax'], nrows=0)
sum_chunks=pd.DataFrame()

bc = pd.read_csv('bookings.sample.csv.bz2',sep='^',usecols=['arr_port','pax'], chunksize=4000)
for i,chunk in enumerate(bc):
    chunk_airports=chunk.dropna().groupby(['arr_port']).sum().reset_index()
    #chunk_airports=chunk.dropna().groupby(['arr_port']).sum().head() #if we dont reset index arr_port is THe index
    #print chunk_airports
    sum_chunks=sum_chunks.append(chunk_airports)
    #sum_chunks=sum_chunks.append(chunk_airports, ignore_index=True)
#sum_chunks


In [11]:
topairports=sum_chunks.groupby('arr_port').pax.sum().sort_values(ascending=False).reset_index().head(10) 
topairports

,arr_port,pax
0,HKG,112
1,LGA,95
2,ORD,94
3,JFK,92
4,SFO,91
5,LAX,91
6,MCO,90
7,DCA,82
8,DEN,79
9,LHR,76


### list of DF

In [276]:
bc = pd.read_csv('bookings.sample.csv.bz2',sep='^',usecols=['arr_port','pax'], chunksize=4000)
tops = []
for b in bc:
    small_df=b.dropna().groupby('arr_port').sum().sort_values('pax',ascending=False).reset_index()
    #print list_top[:10]
    tops.append(small_df)
    
top = pd.concat(tops) 
#top

In [264]:
type(list_top)

pandas.core.frame.DataFrame

In [266]:
type(tops)

list

In [274]:
tops[0].head()

,arr_port,pax
0,DCA,54
1,DEN,49
2,SFO,45
3,PVG,39
4,LAX,39


In [337]:
topairports=top.groupby('arr_port').pax.sum().sort_values(ascending=False).reset_index().head(10) 
topairports

,arr_port,pax
0,HKG,112
1,LGA,95
2,ORD,94
3,JFK,92
4,SFO,91
5,LAX,91
6,MCO,90
7,DCA,82
8,DEN,79
9,LHR,76


### Dictionary

In [507]:
bc = pd.read_csv('bookings.sample.csv.bz2',sep='^',usecols=['arr_port','pax'], chunksize=1000)
#tops_dict = dict()
tops_dict = {}


In [508]:
for b in bc:
    a=b.dropna().groupby('arr_port').sum().sort_values('pax',ascending=False).reset_index()
    for i, row in enumerate(a.iterrows()) :
        airports_name = a.arr_port[i].strip()
        if (not airports_name in tops_dict):
            tops_dict[airports_name]=a.pax[i]
        else:
            tops_dict[airports_name]=tops_dict[airports_name]+a.pax[i]
            

In [512]:
keys=tops_dict.keys()

In [503]:
keys[0]

'CPH'

In [517]:
#tops_dict.items()
#tops_dict.items()[0]
tops_dict.items()[0][1]

18

In [539]:
import operator
sorted_list = sorted(tops_dict.items(), key=operator.itemgetter(1), reverse=True)

In [542]:
 for num in sorted_list[:10]:
        print("    Airport: %s  had %s passengers in 2013" % (num[0], num[1]))

    Airport: CPH  had 18 passengers in 2013
    Airport: DZA  had 4 passengers in 2013
    Airport: OSR  had 0 passengers in 2013
    Airport: AGA  had 0 passengers in 2013
    Airport: HEL  had 3 passengers in 2013
    Airport: PVG  had 49 passengers in 2013
    Airport: SIP  had 4 passengers in 2013
    Airport: YUL  had 14 passengers in 2013
    Airport: AGX  had 2 passengers in 2013
    Airport: CPR  had 3 passengers in 2013


In [504]:
tops=zip(tops_dict.keys(), tops_dict.values())

In [505]:
from collections import Counter
Counter(tops_dict).most_common(10)

[('HKG', 112),
 ('LGA', 95),
 ('ORD', 94),
 ('JFK', 92),
 ('LAX', 91),
 ('SFO', 91),
 ('MCO', 90),
 ('DCA', 82),
 ('DEN', 79),
 ('LHR', 76)]

In [ ]:
for b in bc:
    small_df=b.dropna().groupby('arr_port').sum().sort_values('pax',ascending=False).reset_index()
    #print list_top[:10]
    tops.append(small_df)
    
top = pd.concat(tops) 

### Now we've got top airports

we have to install geobases
pip install GeoBases

In [280]:
from GeoBases import GeoBase

In [284]:
geoDict = GeoBase(data='ori_por', verbose=False)
#geoDict = GeoBase(data='ori_por')

In [285]:
type(geoDict)

GeoBases.GeoBaseModule.GeoBase

In [295]:
#geoDict.keys()

In [289]:
geoDict.fields

['__key__',
 '__dup__',
 '__par__',
 '__lno__',
 'iata_code',
 'icao_code',
 'faa_code',
 'is_geonames',
 'geoname_id',
 'valid_id',
 'name',
 'asciiname',
 'lat',
 'lng',
 'fclass',
 'fcode',
 'page_rank',
 'date_from',
 'date_until',
 'comment',
 'country_code',
 'cc2',
 'country_name',
 'continent_name',
 'adm1_code',
 'adm1_name_utf',
 'adm1_name_ascii',
 'adm2_code',
 'adm2_name_utf',
 'adm2_name_ascii',
 'adm3_code',
 'adm4_code',
 'population',
 'elevation',
 'gtopo30',
 'timezone',
 'gmt_offset',
 'dst_offset',
 'raw_offset',
 'moddate',
 'city_code',
 'city_name_utf',
 'city_name_ascii',
 'tvl_por_list@raw',
 'tvl_por_list',
 'state_code',
 'location_type@raw',
 'location_type',
 'wiki_link',
 'alt_name_section@raw',
 'alt_name_section',
 '__gar__']

In [302]:
geoDict.get?

In [311]:
geoDict.get('LHR','city_name_utf')

'London'

In [315]:
geoDict.get('LHR')['name']

'London Heathrow Airport'

In [317]:
topairports.arr_port[0].strip()

'HKG'

In [343]:
topairports['AirportName']=geoDict.get(topairports.arr_port[0].strip())['name']

In [339]:
topairports

,arr_port,pax,Airport Name
0,HKG,112,Hong Kong International Airport
1,LGA,95,Hong Kong International Airport
2,ORD,94,Hong Kong International Airport
3,JFK,92,Hong Kong International Airport
4,SFO,91,Hong Kong International Airport
5,LAX,91,Hong Kong International Airport
6,MCO,90,Hong Kong International Airport
7,DCA,82,Hong Kong International Airport
8,DEN,79,Hong Kong International Airport
9,LHR,76,Hong Kong International Airport


In [340]:
#topairports['Airport Name']=geoDict.get(topairports.arr_port.strip())['name']
topairports.arr_port

0    HKG     
1    LGA     
2    ORD     
3    JFK     
4    SFO     
5    LAX     
6    MCO     
7    DCA     
8    DEN     
9    LHR     
Name: arr_port, dtype: object

In [341]:
topairports.arr_port.map(lambda x: geoDict.get(x.strip())['name'])

0              Hong Kong International Airport
1                           La Guardia Airport
2         Chicago O'Hare International Airport
3         John F Kennedy International Airport
4          San Francisco International Airport
5            Los Angeles International Airport
6                Orlando International Airport
7    Ronald Reagan Washington National Airport
8                 Denver International Airport
9                      London Heathrow Airport
Name: arr_port, dtype: object

In [370]:
topairports['Airport Name']=topairports.arr_port.map(lambda x: geoDict.get(x.strip())['name'])
topairports

,arr_port,pax,Airport Name
0,HKG,112,Hong Kong International Airport
1,LGA,95,La Guardia Airport
2,ORD,94,Chicago O'Hare International Airport
3,JFK,92,John F Kennedy International Airport
4,SFO,91,San Francisco International Airport
5,LAX,91,Los Angeles International Airport
6,MCO,90,Orlando International Airport
7,DCA,82,Ronald Reagan Washington National Airport
8,DEN,79,Denver International Airport
9,LHR,76,London Heathrow Airport


In [369]:
topairports.drop('AirportName', axis=1, inplace=True)

In [371]:
topairports['AirportName']=geoDict.get(topairports.arr_port[0].strip())['name']

In [372]:
del topairports['AirportName']

In [378]:
topairports

,arr_port,pax,Airport Name
0,HKG,112,Hong Kong International Airport
1,LGA,95,La Guardia Airport
2,ORD,94,Chicago O'Hare International Airport
3,JFK,92,John F Kennedy International Airport
4,SFO,91,San Francisco International Airport
5,LAX,91,Los Angeles International Airport
6,MCO,90,Orlando International Airport
7,DCA,82,Ronald Reagan Washington National Airport
8,DEN,79,Denver International Airport
9,LHR,76,London Heathrow Airport


In [382]:
topairports.drop(topairports.index[[8,3]], axis=0)

ValueError: labels ['arr_port' 'pax' 'Airport Name'] not contained in axis

In [ ]:
topairports.ix[8]

In [388]:
topairports[topairports.pax>90].index

Int64Index([0, 1, 2, 3, 4, 5], dtype='int64')

In [390]:
topairports.drop(topairports[topairports.pax>90].index, axis=0)

,arr_port,pax,Airport Name
6,MCO,90,Orlando International Airport
7,DCA,82,Ronald Reagan Washington National Airport
8,DEN,79,Denver International Airport
9,LHR,76,London Heathrow Airport


In [391]:
next?

In [ ]:
b = pd.read_csv('bookings.csv.bz2',sep='^',usecols=['arr_port','year', 'pax'])

In [406]:
b.groupby('year').count()

,arr_port,pax
year,,
2013,10000009,10000009


In [2]:
a=pd.DataFrame()

In [3]:
type(a)

pandas.core.frame.DataFrame

## Final Solution

In [2]:
import pandas as pd

sum_chunks=pd.DataFrame()

bc = pd.read_csv('bookings.csv.bz2',sep='^',usecols=['arr_port','pax'], chunksize=1000000)
for i,chunk in enumerate(bc):
    print i
    chunk_airports=chunk.dropna().groupby(['arr_port']).sum().reset_index()
    #chunk_airports=chunk.dropna().groupby(['arr_port']).sum().head() #if we dont reset index arr_port is THe index
    #print chunk_airports
    sum_chunks=sum_chunks.append(chunk_airports)
    #sum_chunks=sum_chunks.append(chunk_airports, ignore_index=True)
#sum_chunks
topairports=sum_chunks.groupby('arr_port').pax.sum().sort_values(ascending=False).reset_index().head(10) 
topairports

0
1
2
3
4
5
6
7
8
9
10


,arr_port,pax
0,LHR,88809
1,MCO,70930
2,LAX,70530
3,LAS,69630
4,JFK,66270
5,CDG,64490
6,BKK,59460
7,MIA,58150
8,SFO,58000
9,DXB,55590


## Drop Duplicates()

In [127]:
df=df.append(df.ix[2]).reset_index()
df

,index,one,two,three,four,five
0,a,-1.652355,-0.033970,-0.456528,bar,NaN
1,c,-2.767755,1.406146,-0.771437,bar,NaN
2,e,0.402114,0.759691,0.090970,bar,1
3,f,-0.774120,-0.111994,-0.265381,bar,1
4,h,-0.627316,2.166776,-0.553747,bar,NaN
5,e,0.402114,0.759691,0.090970,bar,1
6,e,0.402114,0.759691,0.090970,bar,1


In [129]:
df.drop_duplicates()

,index,one,two,three,four,five
0,a,-1.652355,-0.033970,-0.456528,bar,NaN
1,c,-2.767755,1.406146,-0.771437,bar,NaN
2,e,0.402114,0.759691,0.090970,bar,1
3,f,-0.774120,-0.111994,-0.265381,bar,1
4,h,-0.627316,2.166776,-0.553747,bar,NaN


In [130]:
df.dropna().drop_duplicates()

,index,one,two,three,four,five
2,e,0.402114,0.759691,0.090970,bar,1
3,f,-0.774120,-0.111994,-0.265381,bar,1


### Compare two DF

In [79]:
b0=pd.read_csv('bookings.csv.bz2',sep='^', nrows=1000)
b1=pd.read_csv('bookings.csv.bz2',sep='^', nrows=1000)
b2=pd.read_csv('bookings.csv.bz2',sep='^', nrows=100)


In [80]:
from pandas.util.testing import assert_frame_equal
assert_frame_equal(b0, b1)

In [81]:
assert_frame_equal(b0, b2)



AssertionError: DataFrame are different

DataFrame shape (number of rows) are different
[left]:  1000, Int64Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
            ...
            990, 991, 992, 993, 994, 995, 996, 997, 998, 999],
           dtype='int64', length=1000)
[right]: 100, Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
            17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
            34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
            51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
            68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
            85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99],
           dtype='int64')

In [83]:
def compare_df (a, b):
    try:
        assert_frame_equal(a, b)
        return True
    except:  # appeantly AssertionError doesn't catch all
        return False

In [84]:
compare_df(b0,b1)

True

In [85]:
compare_df(b0,b2)

False